In [97]:
import os
import json
import numpy as np
import pandas as pd
import re
import csv
import random

In [2]:
path_to_json_files = 'march18_vaccine_twitter/'

In [3]:
json_files = [pos_json for pos_json in os.listdir(path_to_json_files) if pos_json.endswith('.json')]

In [4]:
print(len(json_files))

2288


In [5]:
file_count = 0
tweet_id_text_list = []
tweet_ids = []

for file_name in json_files:
    with open(path_to_json_files + file_name) as json_data:
        data = json.load(json_data)
        
        data_df = pd.DataFrame.from_dict(data)
        
        # Filtering out non-English tweets and replies 
        data_df = data_df[data_df.lang == 'en']
        data_df = data_df[pd.isna(data_df.in_reply_to_screen_name)]
        data_df = data_df[pd.isna(data_df.in_reply_to_status_id)]
        data_df = data_df[pd.isna(data_df.in_reply_to_status_id_str)]
        data_df = data_df[pd.isna(data_df.in_reply_to_user_id)]
        data_df = data_df[pd.isna(data_df.in_reply_to_user_id_str)]
         
        # Iterating over every row of each json file
        for index, row in data_df.iterrows():
            tweet_id = row['id_str']
            
            # Checking if the row doesn't contain a re-tweet, and if the tweet has already been added to the dataset through a re-tweet 
            if pd.isna(row.retweeted_status) and tweet_id not in tweet_ids:                
                try:
                    tweet_text = row['extended_tweet']['full_text']
                except:
                    tweet_text = row['text']
                tweet_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet_text).strip()
                tweet_id_text_list.append([tweet_id, tweet_text])
                tweet_ids.append(tweet_id)
            elif not pd.isna(row.retweeted_status) and tweet_id not in tweet_ids:
                tweet_original_id = row['retweeted_status']['id_str'] 
                
                # Checking if the original tweet has already been added to the dataset
                if tweet_original_id not in tweet_ids:
                    try:
                        tweet_text = row['retweeted_status']['extended_tweet']['full_text']
                    except:
                        tweet_text = row['retweeted_status']['text']
                    tweet_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet_text).strip()
                    tweet_id_text_list.append([tweet_original_id, tweet_text])
                    tweet_ids.append(tweet_original_id)
                    
    file_count += 1
    if file_count % 500 == 0:
        print("Finished: " + str(file_count) + " files")

Finished: 500 files
Finished: 1000 files
Finished: 1500 files
Finished: 2000 files


In [8]:
tweet_df = pd.DataFrame(tweet_id_text_list, columns=['id', 'Text'])

In [7]:
#tweet_df

In [13]:
#tweet_df[tweet_df.duplicated(['Text'], keep=False)]

In [9]:
tweet_df = tweet_df.drop_duplicates(subset='Text', keep="last")

In [10]:
#tweet_df

In [11]:
tweet_df.reset_index(drop=True, inplace=True)

In [12]:
tweet_df

,id,Text
0,969228265901776897,Visit to order FREE resources on critical vac...
1,969151972166328321,March 3 is #WorldBDDay. Abt 8M #babies born wo...
2,969186912031379456,Wakefield's legacy lives on in #measles outbre...
3,969227439808438272,“If you look at the resources that are now ded...
4,969179891097919488,*searching for Caleb gifs and find that time C...
5,969228595855089665,First needle-free live vaccination device hera...
6,968778782931857410,A family in Nangarhar province #Afghanistan is...
7,968906896429277184,"Measles is a very dangerous disease, especiall..."
8,969221732992212993,🇩🇰#Denmark campaign rebuilds confidence in HPV...
9,961768587814989824,My Alicia Patterson fellowship to travel and r...


In [85]:
tweet_df.iloc[41957]["Text"]

'DO NOT GET THE FLU VACCINE!!! Big Pharma wants to make us sick!!!! Use homemade natural flu remedies instead!!\nCDC says 2nd wave of flu virus B has started'

In [206]:
def create_data_batch_csv(df, file_name):
    data_batch = []
    data_batch.append(["Text"])
    for index, row in df.iterrows():
        text_string = row["Text"]
        if text_string[0] == '"':
            text_string = text_string[1:len(text_string)-1]
        
        if text_string[len(text_string)-1] == '"':          
            text_string = text_string[0:len(text_string)-2]
        
        data_batch.append([text_string])
    
    with open(file_name + '.csv', 'w', encoding="utf-8", newline='') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(data_batch)

    csvFile.close()

In [207]:
desired_tweets_count = 200
all_batch_data = tweet_df.sample(n=desired_tweets_count)
batches_count = 10
batches_size = 20

for i in range(0, batches_count):
    batch_i_df = all_batch_data.sample(n=batches_size)
    create_data_batch_csv(batch_i_df, "batch_" + str(i + 1))
    all_batch_data = all_batch_data.drop(batch_i_df.index)

In [208]:
all_batch_data

,id,Text
